# Little Schemer Chapter 06

## The Function Quote

In [2]:
(quote a) ; same as 'a

a

In [10]:
(eq? (quote a) 'a)

#t

In [3]:
(quote +)

+

In [5]:
(eq? (quote +) '+)

#t

### Small Trick: How to get second element of a list?

(car (cdr L)) will do the trick

In [1]:
(car (cdr '(hello OK 88)))

OK

## The Function Numbered?

A function to tell whether a representation of an arithmetic expression contains only numbers besides the + and * (there should be an upper arrow as well).

Don't really understand. May come back later.

## The Function Value

Compute the value of an arithmetic expression of the following form:

1. '3': input is a single number 3
2. 2 + 3: input is a list (2 + 3)
3. 2 + (3 * 5): input is a list (2 + (3 * 5))

The function takes a number or a list as input, where the list contains only numbers and + or * to form arithmetic expressions.

In [10]:
(define length
    (lambda (L)
        (cond
             ((null? L) 0)
             (else (+ 1 (length (cdr L))))
        )
    )
)

In [16]:
(define value
    (lambda (nexp)
        (cond
             ((atom? nexp) nexp)
             ((eq? (length nexp) 1) (value (car nexp)))
             ((eq? (car (cdr nexp)) (quote +)) (+ (value (car nexp)) (value (cdr (cdr nexp)))))
             ((eq? (car (cdr nexp)) (quote *)) (* (value (car nexp)) (value (cdr (cdr nexp)))))
        )
    )
)

In [17]:
(value 88)

88

In [18]:
(value '(2 * 3))

6

In [19]:
(value '(2 + (3 * 5)))

17

In [21]:
(value '(2 + ((2 + 5) * 4)))

30

### Can we simply it?

Actually we can. From the definition, we know that a valid input is either:

1. An atom, or
2. A list of 3 elements, with the second element being either + or *.

If it is case (2), then the first and third element can be either an atom of list. So here comes a simplified version of value, which does not need the 'length' function.

In [22]:
(define value2
    (lambda (nexp)
        (cond
             ((atom? nexp) nexp)
             ((eq? (car (cdr nexp)) (quote +)) (+ (value2 (car nexp)) (value2 (car (cdr (cdr nexp))))))
             ((eq? (car (cdr nexp)) (quote *)) (* (value2 (car nexp)) (value2 (car (cdr (cdr nexp))))))
        )
    )
)

In [23]:
(value2 99)

99

In [24]:
(value2 '(3 * 5))

15

In [25]:
(value2 '(2 + ((2 + 5) * 4)))

30